In [1]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [37]:
import glob

In [63]:
import re

In [42]:
data_list = glob.glob('*_data.csv')
data_list

['6_data.csv',
 '7_data.csv',
 '14_data.csv',
 '1_data.csv',
 '13_data.csv',
 '12_data.csv',
 '963_data.csv',
 '5_data.csv',
 '181_data.csv',
 '8_data.csv',
 '10_data.csv',
 '11_data.csv',
 '88_data.csv']

In [43]:
form_list = list()
for item in data_list:
    form_list.append(item.replace('data', 'form'))
form_list

['6_form.csv',
 '7_form.csv',
 '14_form.csv',
 '1_form.csv',
 '13_form.csv',
 '12_form.csv',
 '963_form.csv',
 '5_form.csv',
 '181_form.csv',
 '8_form.csv',
 '10_form.csv',
 '11_form.csv',
 '88_form.csv']

In [61]:
data_df = pd.DataFrame({'Hit': pd.Series(dtype='int'),
                        'False Alarm': pd.Series(dtype='int'),
                        'No Response': pd.Series(dtype='int'),
                        'Data File': pd.Series(dtype='string')})

for item in data_list:    
    participant_df = pd.read_csv(item)
    hit = 0
    noAnswer = 0
    falseAlarm = 0

    for index, row in participant_df.iterrows():
        if row['Signal'] == 0:
            if row['Response'] == '[]':
                noAnswer += 1
            elif int(row['Response']) == 1:
                hit += 1
        else:
            if row['Response'] == '[]':
                noAnswer += 1
            elif int(row['Response']) == 1:
                falseAlarm += 1
    
    new_row = pd.Series({'Hit': hit,
                        'False Alarm': falseAlarm,
                        'No Response': noAnswer,
                        'Data File': item})
    data_df = pd.concat([data_df, new_row.to_frame().T], ignore_index=True)

    
data_df

,Hit,False Alarm,No Response,Data File
0,7,5,0,6_data.csv
1,10,2,0,7_data.csv
2,12,6,0,14_data.csv
3,6,5,0,1_data.csv
4,10,6,2,13_data.csv
5,10,4,0,12_data.csv
6,6,8,3,963_data.csv
7,10,7,1,5_data.csv
8,13,4,0,181_data.csv
9,8,12,2,8_data.csv


In [65]:
def clean_string(input_string):
    cleaned_string = re.sub(r'[^0-9,]', '', input_string)
    return cleaned_string

In [74]:
def sum_integers(input_string):
    components = input_string.split(',')
    return sum(int(component) for component in components if component.isdigit())

In [77]:
form_df = pd.DataFrame()
for item in form_list:
    new_row = pd.read_csv(item)
    new_row['Form File'] = [item]

    # Processing
    new_row.at[0, 'prof_list'] = clean_string(str(new_row.iloc[0]['prof_list']))
    new_row.at[0, 'sum_proficiency'] = sum_integers(str(new_row.iloc[0]['prof_list']))

    form_df = pd.concat([form_df, new_row], ignore_index=True)

form_df = form_df.drop(form_df.columns[0], axis=1)
form_df

,n_languages,lang_list,prof_list,sum_proficiency,isMono,Form File
0,2,"English, Gujurati","10,5",15,False,6_form.csv
1,3,"english,spanish,Italian","10,4,4",18,True,7_form.csv
2,2,"English, French","10,4",14,False,14_form.csv
3,2,"English, Farsi","10,6",16,True,1_form.csv
4,2,"English, Korean","10,10",20,True,13_form.csv
5,2,"English, German","10,6",16,False,12_form.csv
6,1,English,10,10,True,963_form.csv
7,1,English,10,10,True,5_form.csv
8,2,"Chinese, English","10,9",19,True,181_form.csv
9,4,"English, Spanish, French, Tagalog","10,4,2,3",19,False,8_form.csv


In [78]:
result_df = pd.concat([data_df, form_df], axis=1)
result_df

,Hit,False Alarm,No Response,Data File,n_languages,lang_list,prof_list,sum_proficiency,isMono,Form File
0,7,5,0,6_data.csv,2,"English, Gujurati","10,5",15,False,6_form.csv
1,10,2,0,7_data.csv,3,"english,spanish,Italian","10,4,4",18,True,7_form.csv
2,12,6,0,14_data.csv,2,"English, French","10,4",14,False,14_form.csv
3,6,5,0,1_data.csv,2,"English, Farsi","10,6",16,True,1_form.csv
4,10,6,2,13_data.csv,2,"English, Korean","10,10",20,True,13_form.csv
5,10,4,0,12_data.csv,2,"English, German","10,6",16,False,12_form.csv
6,6,8,3,963_data.csv,1,English,10,10,True,963_form.csv
7,10,7,1,5_data.csv,1,English,10,10,True,5_form.csv
8,13,4,0,181_data.csv,2,"Chinese, English","10,9",19,True,181_form.csv
9,8,12,2,8_data.csv,4,"English, Spanish, French, Tagalog","10,4,2,3",19,False,8_form.csv


In [79]:
result_df.to_csv('result.csv')